In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    """
    Reads assets/wikipedia_data.html and assets/nba.csv (year 2018),
    computes metro-level average W/L% for NBA teams, and returns
    Pearson correlation (r) between metropolitan Population and W/L%.
    Also prints intermediate info and correlation results.
    """
    # --- Helper functions ---
    def safe_int(val):
        if pd.isna(val):
            return np.nan
        s = re.sub(r'[^0-9]', '', str(val))
        return int(s) if s.isdigit() else np.nan

    def split_teams(cell):
        if pd.isna(cell):
            return []
        s = str(cell)
        s = re.sub(r'[/&]', ',', s)
        parts = re.split(r',|\band\b', s)
        return [p.strip() for p in parts if p.strip() not in ['', '—', 'nan']]

    def normalize_team(name):
        if pd.isna(name):
            return np.nan
        s = re.sub(r'\[.*?\]|\(.*?\)', '', str(name))
        s = re.sub(r'[^A-Za-z0-9\s]', '', s)
        s = s.strip().lower()
        if not s:
            return np.nan
        return s.split()[-1]

    TEAM_FIX = {
        'trailblazers': 'blazers',
        'trail blazers': 'blazers',
        'timberwolves': 'wolves',
        'timber wolves': 'wolves',
        'seventysixers': 'sixers',
        '76ers': 'sixers',
        'neworleanspelicans': 'pelicans',
    }

    def canonical(name):
        n = normalize_team(name)
        if pd.isna(n):
            return np.nan
        return TEAM_FIX.get(n, n)

    # --- Read HTML table ---
    tables = pd.read_html("assets/wikipedia_data.html", flavor="lxml")
    table_idx = None
    for i, t in enumerate(tables):
        cols = [str(c).lower() for c in t.columns]
        if any("metropolitan" in c for c in cols) and any("population" in c for c in cols):
            table_idx = i
            break
    if table_idx is None:
        raise RuntimeError("Could not find metropolitan table.")

    cities = tables[table_idx].copy()
    cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']].copy()
    cities.columns = ['Metropolitan area', 'Population', 'NBA']
    cities['NBA'] = cities['NBA'].astype(str).str.replace(r"\[.*\]", "", regex=True)

    # --- Expand team list ---
    rows = []
    for _, r in cities.iterrows():
        metro = r['Metropolitan area']
        pop = safe_int(r['Population'])
        for team in split_teams(r['NBA']):
            rows.append({'Metropolitan area': metro, 'Population': pop, 'team_raw': team})
    team_df = pd.DataFrame(rows)
    team_df['team_norm'] = team_df['team_raw'].apply(canonical)

    # --- Load and clean NBA CSV ---
    nba = pd.read_csv("assets/nba.csv")
    nba = nba[nba['year'] == 2018].copy()
    if 'W/L%' not in nba.columns:
        if 'W-L%' in nba.columns:
            nba.rename(columns={'W-L%': 'W/L%'}, inplace=True)
        elif {'W', 'L'}.issubset(nba.columns):
            nba['W/L%'] = (pd.to_numeric(nba['W'], errors='coerce') /
                           (pd.to_numeric(nba['W'], errors='coerce') +
                            pd.to_numeric(nba['L'], errors='coerce')))
    nba['W/L%'] = pd.to_numeric(nba['W/L%'], errors='coerce')
    if nba['W/L%'].max(skipna=True) > 1:
        nba['W/L%'] = nba['W/L%'] / 100.0
    nba['team_norm'] = nba['team'].apply(canonical)

    # Manual metro fixes
    manual_team_to_metro = {
        'warriors': 'San Francisco Bay Area',
        'nets': 'New York City',
        'knicks': 'New York City',
        'clippers': 'Los Angeles',
        'lakers': 'Los Angeles'
    }
    nba['team_metro'] = nba['team_norm'].map(manual_team_to_metro)

    # --- Merge and group ---
    merged = pd.merge(team_df[['Metropolitan area', 'Population', 'team_norm']],
                      nba[['team_norm', 'team_metro', 'W/L%']],
                      on='team_norm', how='inner')
    merged['Metropolitan area'] = merged['team_metro'].combine_first(merged['Metropolitan area'])
    grouped = merged.groupby('Metropolitan area').agg({'W/L%': 'mean', 'Population': 'mean'}).dropna()

    # --- Compute correlation ---
    pop = grouped['Population'].astype(float)
    win_loss = grouped['W/L%'].astype(float)
    corr, pval = stats.pearsonr(pop, win_loss)

    # --- Print results ---
    print(f"Found {len(tables)} tables in assets/wikipedia_data.html")
    print(f"Table columns: {cities.columns.tolist()}\n")
    print(f"Number of metros analyzed: {len(grouped)}")
    print(grouped.head(10))
    print(f"\n✅ Pearson correlation between population and NBA win/loss ratio (2018): {corr:.4f}")
    print(f"p-value: {pval:.4f}")

    return corr

# Run automatically when this script is executed
if __name__ == "__main__":
    nba_correlation()


C:\Users\Justin\AppData\Local\Temp\ipykernel_11380\513890407.py:55: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html("assets/wikipedia_data.html", flavor="lxml")


ImportError: Missing optional dependency 'lxml'.  Use pip or conda to install lxml.